In [ ]:
# Imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr # Gradio for creating a web-based interface

# Load environment variables from a .env file to store API keys securely.

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

# Print the key prefixes to verify if they are loaded correctly (helps with debugging)
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")
    
# Initialize API clients for OpenAI, Anthropic (Claude), and Google Gemini
openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

# Define a system message to ensure consistent assistant behavior.

system_message = "You are a helpful assistant that responds in markdown"

## User interface

In [ ]:
def stream_gpt(prompt):
    """
    Stream responses from OpenAI's GPT-4o-mini model.
    """
    messages = [
        {'role': 'system', 'content':system_message},
        {'role': 'user', 'content': prompt}
    ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ''
    for chunk in stream:
        result += chunk.choices[0].delta.content or ''
        yield result
        
def stream_claude(prompt):
    """
    Stream responses from Anthropic's Claude 3 Haiku model.
    """
    result = claude.messages.stream(
        model='claude-3-haiku-20240307',
        max_tokens=1000,
        temperature=0.5,
        system=system_message,
        messages=[
            {'role': 'user', 'content': prompt},
        ]
    )
    response = ''
    with result as stream:
        for text in stream.text_stream:
            response += text or ''
            yield response
            
def stream_gemini(prompt):
    """
    Stream responses from Google's Gemini 2.0 Flash model.
    """
    gemini = google.generativeai.GenerativeModel(
        model_name='gemini-2.0-flash-exp',
        system_instruction=system_message
    )
    stream = gemini.generate_content(prompt, stream=True)
    result = ''
    for chunk in stream:
        result += chunk.text or ''
        yield result

# Dictionary to map model names to their respective streaming functions
MODEL_FUNCTIONs = {
    'GPT': stream_gpt,
    'Claude': stream_claude,
    'Gemini': stream_gemini
}        

def stream_model(prompt, model):
    """
    Call the appropriate model function based on user selection.
    """
    if model not in MODEL_FUNCTIONs:
        raise ValueError('Unknown model')
    yield from MODEL_FUNCTIONs[model](prompt)

# Create a simple Gradio interface for user interaction.
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label='Your message:'),   # User input field
            gr.Dropdown(['GPT', 'Claude', 'Gemini'], label='Select model')],   # Model selection dropdown
    outputs=[gr.Markdown(label='Response:')],    # Display response in markdown format
    flagging_mode='never'    # Disable flagging
)

# Launch the web interface.
view.launch()